In [56]:
import torch
import pandas as pd

In [57]:
class Neuron:
    def __init__(self, num_inputs, num_outputs):
        self.w = torch.randn(num_inputs, num_outputs, requires_grad=True, dtype=torch.float32)
        self.b = torch.randn(num_outputs, 1, requires_grad=True, dtype=torch.float32)

    def forward(self, inputs):
        self.out = (inputs @ self.w + self.b).relu()
        return self.out

    def train(self, inputs, outputs, epochs, learning_rate):
        for i in range(epochs):
            loss = ((outputs - self.forward(inputs))**2).flatten().sum()
            loss.backward()
            print(f"loss: {loss.data}")
            self.w.data = self.w.data - self.w.grad * learning_rate 
            self.b.data = self.b.data - self.b.grad * learning_rate 
            self.w.grad = None
            self.b.grad = None
            


In [58]:
class Layer:
    def __init__(self, num_neurons, inputs_per_neuron): 
        self.num_neurons = num_neurons
        self.w = torch.randn(num_neurons, inputs_per_neuron, requires_grad=True, dtype=torch.float32)
        self.b = torch.randn(num_neurons, 1, requires_grad=True, dtype=torch.float32)
        

    def forward(self, inputs):
        out = ((self.w @ inputs) + self.b)
        return out 
        

In [59]:
class MLP:
    def __init__(self, num_layers, num_inputs, num_neurons_per_layer, num_final_out):
        self.num_layers = num_layers
        self.layers = [Layer(num_neurons_per_layer, num_inputs)]

        for _ in range(num_layers): 
            layer = Layer(num_neurons_per_layer, num_neurons_per_layer)
            self.layers.append(layer)
        
        layer_out = Layer(num_final_out, num_neurons_per_layer)
        self.layers.append(layer_out)
        
    def forward(self, input):
        out_prev = input
        for i in range(len(self.layers)):
            if i == (len(self.layers)-1):
                out_prev = self.layers[i].forward(out_prev)
            else:
                out_prev = self.layers[i].forward(out_prev).tanh()
        return out_prev
        
    def train(self, epochs, learning_rate, x, y):
        loss_func = torch.nn.CrossEntropyLoss()
        for _ in range(epochs):
            # loss = ((y - self.forward(x))**2).flatten().sum()
            y_pred = self.forward(x).view(y.shape[0], y.shape[1])
            # print(y_pred.shape, y_pred.dtype)
            loss = loss_func(y_pred, y)
            print(f"Loss: {loss}")
            loss.backward()
            for layer in self.layers:
                layer.w.data -= learning_rate * layer.w.grad
                layer.b.data -= learning_rate * layer.b.grad
                layer.w.grad = None
                layer.b.grad = None
            
        

In [67]:
#Hyperparameters
epochs = 1000
learning_rate = 0.05
num_neurons_per_layer = 16
num_layers = 2
train_split = 0.90
batch_size = int(150*train_split)

In [61]:
def datagen(batch_size):
    headers=["sepal length", "sepal width", "petal length", "petal width", "name"]
    data = pd.read_csv("/Users/adityatandon/Documents/VS Code/Learn ML/Data/iris.data", names=headers).sample(frac=1).reset_index(drop=True)

    dataset_size = len(data) # 150
    num_classifications = len(set(data.name)) # 3
    num_features = len(data.columns) - 1 # 4

    rand_seed_idx = torch.randint((dataset_size - batch_size), (1,)).item() #generates a tensor with a random integer and extracts it using .item()
    training_data = data[rand_seed_idx : rand_seed_idx+batch_size].reset_index(drop=True, inplace=False)

    diff_flowers = set(data.name)
    classification = []
    for idx in range(len(training_data['name'])):
        if training_data.name[idx] == "Iris-setosa":
            classification.append([1,0,0])
        elif training_data.name[idx] == "Iris-versicolor":
            classification.append([0,1,0])
        elif training_data.name[idx] == "Iris-virginica":
            classification.append([0,0,1])
    
    training_data['classification'] = classification
    features_list=[]
    for i in range(num_features):
        features_list.append(torch.tensor(training_data.iloc(axis=1)[i].tolist()).reshape(len(training_data.iloc(axis=1)[0].tolist()), 1)) 

    train_in = torch.cat(features_list, dim=1)
    train_out = torch.tensor(training_data['classification'].tolist(), dtype=torch.float32)
    train_in = train_in.reshape(batch_size, num_features, 1)
    train_out = train_out.reshape(batch_size, num_classifications)

    examples = train_split * dataset_size
    num_inputs = num_features
    num_outputs = num_classifications

    return train_in, train_out, num_features, num_classifications

In [62]:
train_in, train_out, num_features, num_classifications = datagen(batch_size)

In [63]:
classifier = MLP(num_layers=3, num_inputs=num_features, num_neurons_per_layer=num_neurons_per_layer, num_final_out=num_classifications)

In [64]:
out = classifier.forward(train_in).view(train_out.shape[0], train_out.shape[1])

In [65]:
out[0], train_out.shape

(tensor([-0.8150, -1.9397,  8.4740], grad_fn=<SelectBackward0>),
 torch.Size([135, 3]))

In [68]:
classifier.train(epochs, learning_rate, train_in, train_out)

Loss: 4.882198333740234
Loss: 2.6204450130462646
Loss: 4.033931732177734
Loss: 1.055057406425476
Loss: 0.9541016817092896
Loss: 0.8764929175376892
Loss: 0.8208673596382141
Loss: 0.8105874061584473
Loss: 0.6726351976394653
Loss: 0.6398022770881653
Loss: 0.5880043506622314
Loss: 0.5603414177894592
Loss: 0.5320409536361694
Loss: 0.5161989331245422
Loss: 0.5053256750106812
Loss: 0.4984767735004425
Loss: 0.4944321811199188
Loss: 0.49199172854423523
Loss: 0.4903290867805481
Loss: 0.48905205726623535
Loss: 0.48800086975097656
Loss: 0.48710253834724426
Loss: 0.48631545901298523
Loss: 0.48561254143714905
Loss: 0.48497501015663147
Loss: 0.4843895137310028
Loss: 0.48384609818458557
Loss: 0.48333752155303955
Loss: 0.4828581213951111
Loss: 0.48240339756011963
Loss: 0.4819701015949249
Loss: 0.4815555810928345
Loss: 0.4811572730541229
Loss: 0.4807736873626709
Loss: 0.48040324449539185
Loss: 0.4800444543361664
Loss: 0.47969648241996765
Loss: 0.4793583154678345
Loss: 0.47902911901474
Loss: 0.4787081182